# Classification Surrogate Tests

We are interested in testing whether or not a surrogate model can correctly identify unknown constraints based on categorical criteria with classification surrogates. Essentially, we want to account for scenarios where specialists can look at a set of experiments and label outcomes as 'acceptable', 'unacceptable', 'ideal', etc. 

This involves new models that produce `CategoricalOutput`'s rather than continuous outputs. Mathematically, if $g_{\theta}:\mathbb{R}^d\to[0,1]^c$ represents the function governed by learnable parameters $\theta$ which outputs a probability vector over $c$ potential classes (i.e. for input $x\in\mathbb{R}^d$, $g_{\theta}(x)^\top\mathbf{1}=1$ where $\mathbf{1}$ is the vector of all 1's) and we have acceptibility criteria for the corresponding classes given by $a\in\{0,1\}^c$, we can compute the scalar output $g_{\theta}(x)^\top a\in[0,1]$ which represents the expected value of acceptance as an objective value to be passed in as a constrained function.

In this script, we look at the [Rosenbrock function constrained to a disk](https://en.wikipedia.org/wiki/Test_functions_for_optimization#cite_note-12) which attains a global minima at $(x_0^*,x_1^*)=(1.0, 1.0)$. To facilitate testing the functionality offered by BoFire, we label all points inside of the circle $x_0^2+x_1^2\le2$ as 'acceptable' and futher label anything inside of the interesction of this circle and the circle $(x_0-1)^2+(x_1-1)^2\le1.0$ as 'ideal'; points lying outside of these two locations are labeled as "unacceptable."

In [1]:
# Import packages
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain, Outputs, Inputs
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput, CategoricalOutput, CategoricalInput
from bofire.data_models.objectives.api import MinimizeObjective, MinimizeSigmoidObjective, ConstrainedCategoricalObjective
import numpy as np
import pandas as pd

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Manual setup of the optimization domain

The following cells show how to manually setup the optimization problem in BoFire for didactic purposes.

In [2]:
# Write helper functions which give the objective and the constraints
def rosenbrock(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    return (1 - x["x_0"]) ** 2 + 100 * (x["x_1"] - x["x_0"] ** 2) ** 2

def constraints(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    feasiblity_vector = []
    for _, row in x.iterrows():
        if (row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0) and ((row["x_0"] - 1.0) ** 2 + (row["x_1"] - 1.0) ** 2 <= 1.0):
            feasiblity_vector.append("ideal")
        elif row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0:
            feasiblity_vector.append("acceptable")
        else:
            feasiblity_vector.append("unacceptable")
    return feasiblity_vector

In [3]:
# Set-up the inputs and outputs, use categorical domain just as an example
input_features = Inputs(features=[ContinuousInput(key=f"x_{i}", bounds=(-1.75, 1.75)) for i in range(2)] + [CategoricalInput(key=f"x_3", categories=["0", "1"], allowed=[True, True])])

# here the minimize objective is used, if you want to maximize you have to use the maximize objective.
output_features = Outputs(features=[
        ContinuousOutput(key=f"f_{0}", objective=MinimizeObjective(w=1.)),
        CategoricalOutput(key=f"f_{1}", categories=["unacceptable", "acceptable", "ideal"], objective=ConstrainedCategoricalObjective(categories=["unacceptable", "acceptable", "ideal"], desirability=[False, True, True])), # This function will be associated with learning the categories
        ContinuousOutput(key=f"f_{2}", objective=MinimizeSigmoidObjective(w=1., tp=0.0, steepness=0.5)),
    ]
)

# Create domain
domain1 = Domain(inputs=input_features, outputs=output_features)

# Sample random points
sample_df = domain1.inputs.sample(100)

# Write a function which outputs one continuous variable and another discrete based on some logic
sample_df["f_0"] = rosenbrock(x=sample_df)
sample_df["f_1"] = constraints(x=sample_df)
sample_df["f_2"] = sample_df["x_3"].astype(float) + 1e-2 * np.random.uniform(size=(len(sample_df),))
sample_df.head(5)

,x_0,x_1,x_3,f_0,f_1,f_2
0,-0.350619,0.504052,1,16.349300,acceptable,1.009714
1,0.115844,0.947509,0,88.034069,ideal,0.005132
2,-0.556177,-0.459197,1,61.485379,acceptable,1.000490
3,-1.635584,0.905708,1,320.033865,unacceptable,1.006897
4,-1.474244,0.855846,0,179.715811,unacceptable,0.005518


In [4]:
# Plot the sample df
import math
import plotly.express as px 
fig = px.scatter(sample_df, x="x_0", y="x_1", color="f_1", width=550, height=525, title="Samples with labels")
fig.add_shape(type="circle",
    xref="x", yref="y",
    opacity=0.1,
    fillcolor="red",
    x0=-math.sqrt(2), y0=-math.sqrt(2), x1=math.sqrt(2), y1=math.sqrt(2),
    line_color="red",
)
fig.add_shape(type="circle",
    xref="x", yref="y",
    opacity=0.2,
    fillcolor="LightSeaGreen",
    x0=0, y0=0, x1=2, y1=2,
    line_color="LightSeaGreen",
)
fig.show()

## Evaluate the classification model performance (outside of the optimization procedure)

In [5]:
# Import packages
import bofire.surrogates.api as surrogates
from bofire.data_models.surrogates.api import ClassificationMLPEnsemble
from bofire.surrogates.diagnostics import ClassificationMetricsEnum

# Instantiate the surrogate model 
model = ClassificationMLPEnsemble(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]), lr=0.03, n_epochs=100, hidden_layer_sizes=(4,2,), weight_decay=0.0, batch_size=10, activation="tanh")
surrogate = surrogates.map(model)

# Fit the model to the classification data
cv_df = sample_df.drop(["f_0", "f_2"], axis=1)
cv_df["valid_f_1"] = 1
cv = surrogate.cross_validate(cv_df, folds=3)


c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\bo

In [6]:
# Print results
cv[0].get_metrics(metrics=ClassificationMetricsEnum, combine_folds=True) # print training set performance

,ACCURACY,F1
0,0.72,0.72


In [7]:
cv[1].get_metrics(metrics=ClassificationMetricsEnum, combine_folds=True) # print test set performance

,ACCURACY,F1
0,0.65,0.65


## Setup strategy and ask for candidates



In [8]:
from bofire.data_models.acquisition_functions.api import qEI
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.surrogates.api import BotorchSurrogates, ClassificationMLPEnsemble, MixedSingleTaskGPSurrogate
from bofire.data_models.domain.api import Outputs

strategy_data = SoboStrategy(domain=domain1, 
                             acquisition_function=qEI(), 
                             surrogate_specs=BotorchSurrogates(surrogates=
                                    [
                                        ClassificationMLPEnsemble(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]), lr=0.03, n_epochs=100, hidden_layer_sizes=(4,2,), weight_decay=0.0, batch_size=10, activation="tanh"),
                                        MixedSingleTaskGPSurrogate(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs.get_by_key("f_2")]))
                                    ]
                                )
                            )

strategy = strategies.map(strategy_data)

strategy.tell(sample_df)

In [9]:
candidates = strategy.ask(10)
candidates

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:230: RuntimeWarning:

Could not update `train_inputs` with transformed inputs since _MLPEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.



,x_0,x_1,x_3,f_1_pred,f_1_sd,f_1_unacceptable_prob,f_1_acceptable_prob,f_1_ideal_prob,f_0_pred,f_2_pred,f_1_unacceptable_sd,f_1_acceptable_sd,f_1_ideal_sd,f_0_sd,f_2_sd,f_0_des,f_2_des,f_1_des
0,0.154314,-0.001118,1,acceptable,0.0,0.425532,1.004986,0.639645,0.357941,0.002414,6.588467,0.003142,0.476779,0.473279,0.004336,-0.357941,0.499698,1.644631
1,0.149196,0.013251,0,ideal,0.0,0.465530,0.005402,0.795662,0.204113,0.000226,3.777311,0.003156,0.430231,0.430007,0.000242,-0.204113,0.499972,0.801064
2,-1.333767,1.750000,0,ideal,0.0,0.623769,0.007608,0.795045,0.204738,0.000217,11.252150,0.003407,0.431594,0.431392,0.000222,-0.204738,0.499973,0.802653
3,-0.630759,0.390813,1,unacceptable,0.0,2.985411,1.005220,0.638927,0.359041,0.002033,6.575972,0.003180,0.478100,0.475209,0.003490,-0.359041,0.499746,1.644147
4,-1.114157,1.209882,0,unacceptable,0.0,8.030368,0.006634,0.795048,0.204736,0.000216,14.069044,0.003171,0.431596,0.431394,0.000222,-0.204736,0.499973,0.801682
5,-0.358540,0.102376,1,unacceptable,0.0,1.546216,1.004791,0.638009,0.360123,0.001868,4.052824,0.003159,0.479443,0.476816,0.003129,-0.360123,0.499766,1.642799
6,0.736046,0.516076,1,acceptable,0.0,-0.240881,1.004885,0.819717,0.173096,0.007188,8.248212,0.003162,0.331533,0.333793,0.014980,-0.173096,0.499102,1.824602
7,-0.558970,0.322902,0,unacceptable,0.0,2.070167,0.005272,0.795045,0.204738,0.000217,4.657032,0.003146,0.431604,0.431400,0.000224,-0.204738,0.499973,0.800317
8,0.303819,0.044806,1,acceptable,0.0,0.656479,1.005106,0.644744,0.351593,0.003664,6.172526,0.003141,0.468581,0.463146,0.007114,-0.351593,0.499542,1.649849
9,-0.992659,0.943164,0,unacceptable,0.0,6.008622,0.006171,0.795048,0.204736,0.000216,10.497928,0.003149,0.431597,0.431395,0.000222,-0.204736,0.499973,0.801219


## Check classification of proposed candidates

Use the logic from above to verify the classification values

In [10]:
# Append to the candidates
candidates["f_1_true"] = constraints(x=candidates)

In [12]:
# Print results
candidates[["x_0", "x_1", "f_1_pred", "f_1_true"]]

,x_0,x_1,f_1_pred,f_1_true
0,0.154314,-0.001118,acceptable,acceptable
1,0.149196,0.013251,ideal,acceptable
2,-1.333767,1.750000,ideal,unacceptable
3,-0.630759,0.390813,unacceptable,acceptable
4,-1.114157,1.209882,unacceptable,unacceptable
5,-0.358540,0.102376,unacceptable,acceptable
6,0.736046,0.516076,acceptable,ideal
7,-0.558970,0.322902,unacceptable,acceptable
8,0.303819,0.044806,acceptable,acceptable
9,-0.992659,0.943164,unacceptable,acceptable
